In [12]:
from FORM import FORM, PROP
from cnf_io import read_cnf_file
from variable import reset_calls, increment_calls, value_calls
from sat_solver import naive_solve, tree_solve_random, tree_solve
import threading
import matplotlib.pyplot as plt
import random
import time
import csv
import func_timeout
import os


create code to randomly add cnf clauses to file

In [3]:
def add_clauses(clauses, file_path):
  """
  Adds clauses to a dimacs cnf file.

  Args:
    clauses: number of 3-cnf clauses to add
    file_path: path to the file to add clauses to
  """

  # Read the entire CNF file into memory.
  with open(file_path, "r") as f:
    lines = f.readlines()

  # Get the number of variables and clauses in the CNF file.
  first_line = lines[0]
  words = first_line.split()
  num_vars = int(words[2])
  num_clauses = int(words[3])

  # Update the number of clauses in the first line of the CNF file.
  num_clauses += clauses
  updated_first_line = "p cnf {} {}\n".format(num_vars, num_clauses)

  # Replace the original first line with the updated first line.
  lines[0] = updated_first_line

  # Add the new clauses to the CNF file in memory.
  for i in range(clauses):
    # Generate a random 3-clause.
    clause = [random.randint(-num_vars, num_vars) for _ in range(3)]

    # Check for zeros and replace with non-zeros.
    for j in range(3):
      if clause[j] == 0:
        clause[j] = 1 if random.random() < 0.5 else -1

    # Add the clause to the CNF file in memory.
    lines.append(" ".join(map(str, clause)) + " 0\n")

  # Write the updated CNF file back to the file system.
  with open(file_path, "w") as f:
    f.writelines(lines)


In [4]:
add_clauses(500,'Experiment/example_1_file.cnf')

In [5]:
def create_cnf(prop_num, clause_num, file_name):
  """Creates a CNF file with the given number of variables and clauses.

  Args:
    prop_num: The number of variables in the CNF file.
    clause_num: The number of clauses in the CNF file.
    file_name: The name of the CNF file to create.
  """

  # Create the folder if it doesn't exist
  folder_path = os.path.dirname(file_name)
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  # Create the CNF file
  with open(file_name, "w") as f:
    # Write the problem line in the DIMACS format
    f.write(f"p cnf {prop_num} {clause_num}\n")

    # Add random clauses
    for i in range(clause_num):
      clause = [random.randint(-prop_num, prop_num) for _ in range(3)]

      # Check for zeros and replace with non-zeros
      for j in range(3):
        if clause[j] == 0:
          clause[j] = 1 if random.random() < 0.5 else -1

      # Add clause to file
      f.write(" ".join(map(str, clause)) + " 0\n")


In [6]:
create_cnf(50,50,'Experiment/example_1_file.cnf')

run an experiment that compares the different heuristics over differing numbers of clauses and propositions. Experiment will initially be run for 3-cnf formulas and be saved to text files to keep the data safe while it is running.

create experiment for no heuristic solve

In [7]:
print(f"cnf_3_clauses_{10}_props_{5}.cnf")

cnf_3_clauses_10_props_5.cnf


In [8]:
def output_tuple_to_csv(tuple, csv_file_name):
  """Outputs a tuple to a CSV file, creating the file and folder if necessary.

  Args:
    tuple: A tuple of strings to be output to the CSV file.
    csv_file_name: The name of the CSV file to output to.
  """

  # Create the folder if it doesn't exist
  folder_path = os.path.dirname(csv_file_name)
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  # Create the CSV file if it doesn't exist
  if not os.path.exists(csv_file_name):
    with open(csv_file_name, "w", newline="") as f:
      writer = csv.writer(f)

  # Open the CSV file in append mode
  with open(csv_file_name, "a", newline="") as f:
    writer = csv.writer(f)

    # Write the tuple to the CSV file
    writer.writerow(tuple)

In [9]:
reset_calls()
increment_calls()
increment_calls()
increment_calls()
value_calls()
reset_calls()
value_calls()
increment_calls()
value_calls()

1

In [10]:
create_cnf(5,11,"Experiment/formulas/cnf_3_props_11.cnf")
formula = read_cnf_file("Experiment/formulas/cnf_3_props_11.cnf")
reset_calls()
tree_solve(formula)
value_calls()

7

In [11]:
output_tuple_to_csv(('file name', 'propositions', 'clauses','solve time'),'Experiment/experiment_testing.csv')

In [15]:
prop_original = 40
test_num = 1
prop_num = prop_original
clause_num = int(prop_num * 3)
formula_file_name = f"Experiment/formulas/cnf_3_testing.cnf"
data_file_name='Experiment/Data/experiment_unit_preference.csv'
#output_tuple_to_csv(('propositions', 'clauses', 'result','DPLL calls','L/N ratio','solve time'),data_file_name)
for i in range(16):
    print(i)
    LN_ratio = (3+i*0.2)
    for _ in range(100):
        clause_num = int(prop_num * LN_ratio)
        create_cnf(prop_num,clause_num,formula_file_name)
        formula = read_cnf_file(formula_file_name)        
        try:
            reset_calls()
            start = time.time()
            solution = func_timeout.func_timeout(tree_solve(formula, unit_preference=True), 30)
            end = time.time()
            calls = value_calls()
            elapsed_time = end - start
        except TimeoutError:
            prop_num -= 5
            clause_num = int(prop_num * LN_ratio)
            continue
        if solution != 'UNSAT' and not(formula.evaluate(solution)):
            output_tuple_to_csv(('****WRONG****'),data_file_name)
        else:
            output_tuple_to_csv((prop_num,clause_num, formula.evaluate(solution), calls, LN_ratio ,elapsed_time),data_file_name)
            prop_num += 2
            clause_num = int(prop_num * LN_ratio)

    


    

0


TypeError: '>' not supported between instances of 'int' and 'dict'

In [29]:
prop_original = 40
test_num = 1
prop_num = prop_original
clause_num = int(prop_num * 3)
formula_file_name = f"Experiment/formulas/cnf_3_testing.cnf"
data_file_name='Experiment/Data/experiment_unit_preference.csv'
#output_tuple_to_csv(('propositions', 'clauses', 'result','DPLL calls','L/N ratio','solve time'),data_file_name)
for i in range(16):
    print(i)
    LN_ratio = (3+i*0.2)
    for _ in range(100):
        clause_num = int(prop_num * LN_ratio)
        create_cnf(prop_num,clause_num,formula_file_name)
        formula = read_cnf_file(formula_file_name) 
        reset_calls()
        start = time.time()       
        try:
            solution = func_timeout.func_timeout(30, tree_solve, [formula, {"unit_preference": True}])
        except TimeoutError:
            prop_num -= 5
            clause_num = int(prop_num * LN_ratio)
            continue
        if solution != 'UNSAT' and not(formula.evaluate(solution)):
            output_tuple_to_csv(('****WRONG****'),data_file_name)
        else:
            end = time.time()
            calls = value_calls()
            elapsed_time = end - start
            output_tuple_to_csv((prop_num,clause_num, formula.evaluate(solution), calls, LN_ratio ,elapsed_time),data_file_name)
            prop_num += 2
            clause_num = int(prop_num * LN_ratio)

0


FunctionTimedOut: Function tree_solve (args=[((x_64 v x_31 v x_72) ^ ((-x_125) v x_130 v (-x_95)) ^ ((-x_2) v (-x_85) v (-x_96)) ^ ((-x_82) v (-x_40) v (-x_13)) ^ (x_134 v x_52 v (-x_46)) ^ ((-x_85) v x_15 v x_138) ^ ((-x_107) v (-x_85) v x_79) ^ ((-x_20) v x_131 v x_38) ^ ((-x_55) v (-x_20) v (-x_17)) ^ (x_47 v (-x_44) v x_68) ^ ((-x_125) v (-x_41) v x_3) ^ (x_61 v x_29 v (-x_108)) ^ (x_76 v (-x_41) v x_44) ^ (x_1 v x_22 v (-x_86)) ^ (x_97 v (-x_114) v (-x_36)) ^ ((-x_57) v (-x_46) v x_7) ^ ((-x_6) v x_58 v x_10) ^ (x_41 v (-x_129) v x_34) ^ (x_89 v (-x_124) v (-x_109)) ^ (x_46 v x_61 v (-x_46)) ^ (x_57 v (-x_30) v x_132) ^ (x_57 v x_129 v (-x_60)) ^ (x_136 v x_31 v (-x_110)) ^ (x_49 v (-x_71) v x_135) ^ (x_121 v x_36 v (-x_64)) ^ ((-x_121) v x_94 v x_28) ^ (x_63 v x_3 v x_7) ^ (x_26 v x_31 v x_131) ^ ((-x_94) v (-x_10) v x_14) ^ (x_75 v x_76 v x_120) ^ (x_26 v x_30 v x_138) ^ ((-x_26) v x_27 v (-x_137)) ^ ((-x_83) v (-x_138) v x_98) ^ (x_18 v x_87 v x_75) ^ (x_126 v (-x_119) v x_110)

In [28]:
formula = read_cnf_file(formula_file_name)
solution = func_timeout.func_timeout(30, tree_solve, [formula, {"unit_preference": True}])